In [1]:
from data.utils import download_and_save_file_if_needed, save_download_data_to_path
from pathlib import Path
import os

# get closest airports
parent_directory = Path(os.path.abspath("")).parent / "data"

airports_data_directory = parent_directory / "airports"

url_to_download = "https://ourairports.com/data/airports.csv"
file_path_to_save = Path(airports_data_directory) / "airports.csv"
download_and_save_file_if_needed(url_to_download, file_path_to_save)

print("Completed downloading airport data")

Completed downloading airport data


In [2]:
import pandas as pd
import geopandas as gpd

airport_data = Path(airports_data_directory) / "airports.csv"

# load airport data into dataframe, keys to use are medium_airport and large_airport, not much else seems relevant
airports_df = pd.read_csv(airport_data)
filtered_airports_df = airports_df.query("type in ('medium_airport', 'large_airport')")

filtered_airports_gpd = gpd.GeoDataFrame(
    filtered_airports_df,
    geometry=gpd.points_from_xy(filtered_airports_df.longitude_deg, filtered_airports_df.latitude_deg),
)
filtered_airports_gpd

/Users/madhu/PycharmProjects/aptible_geodata/.venv/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,geometry
6451,12243,5A8,medium_airport,Aleknagik / New Airport,59.282600,-158.617996,66.0,NaN,US,US-AK,Aleknagik,yes,5A8,WKK,5A8,NaN,https://en.wikipedia.org/wiki/Aleknagik_Airport,NaN,POINT (-158.61800 59.28260)
11054,332240,AF-0005,medium_airport,Khost International Airport,33.284605,69.807340,4204.0,AS,AF,AF-KHO,Khost,no,OAKS,KHT,NaN,NaN,NaN,NaN,POINT (69.80734 33.28460)
11090,3,AGGH,large_airport,Honiara International Airport,-9.428000,160.054993,28.0,OC,SB,SB-GU,Honiara,yes,AGGH,HIR,NaN,NaN,https://en.wikipedia.org/wiki/Honiara_Internat...,Henderson Field,POINT (160.05499 -9.42800)
11095,4,AGGM,medium_airport,Munda Airport,-8.327970,157.263000,10.0,OC,SB,SB-WE,Munda,yes,AGGM,MUA,NaN,NaN,https://en.wikipedia.org/wiki/Munda_Airport,NaN,POINT (157.26300 -8.32797)
11118,312371,AHJ,medium_airport,Hongyuan Airport,32.531540,102.352240,11600.0,AS,CN,CN-51,Aba,no,ZUHY,AHJ,NaN,NaN,https://en.wikipedia.org/wiki/Hongyuan_Airport,NaN,POINT (102.35224 32.53154)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71987,307013,ZYTN,medium_airport,Tonghua Sanyuanpu Airport,42.253889,125.703333,1200.0,AS,CN,CN-22,Tonghua,yes,ZYTN,TNH,NaN,NaN,https://en.wikipedia.org/wiki/Tonghua_Sanyuanp...,Tonghua Liuhe Airport,POINT (125.70333 42.25389)
71988,27243,ZYTX,large_airport,Shenyang Taoxian International Airport,41.639801,123.483002,198.0,AS,CN,CN-21,"Hunnan, Shenyang",yes,ZYTX,SHE,NaN,NaN,https://en.wikipedia.org/wiki/Shenyang_Taoxian...,NaN,POINT (123.48300 41.63980)
71990,27244,ZYYJ,medium_airport,Yanji Chaoyangchuan Airport,42.882801,129.451004,624.0,AS,CN,CN-22,Yanji,yes,ZYYJ,YNJ,NaN,NaN,https://en.wikipedia.org/wiki/Yanji_Chaoyangch...,NaN,POINT (129.45100 42.88280)
71991,317861,ZYYK,medium_airport,Yingkou Lanqi Airport,40.542524,122.358600,NaN,AS,CN,CN-21,"Laobian, Yingkou",yes,ZYYK,YKH,NaN,NaN,https://en.wikipedia.org/wiki/Yingkou_Lanqi_Ai...,NaN,POINT (122.35860 40.54252)


In [3]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.ops import nearest_points

from data.locations import raw_locations

# find closest airport to a given location
locations_with_closest_airport = pd.DataFrame(data=None, columns=["location", "closest_airport_id"])
for location in raw_locations:
    closest = filtered_airports_gpd.distance(Point((location[0], location[1]))).sort_values().index[0]
    locations_with_closest_airport.loc[len(locations_with_closest_airport)] = [
        (location[0], location[1]),
        filtered_airports_gpd.loc[closest].id,
    ]

locations_with_closest_airport

,location,closest_airport_id
0,"(-77.1528, 39.085)",3483
1,"(-74.6551, 40.3431)",3929
2,"(-74.6551, 40.3431)",3929
3,"(-71.0589, 42.3601)",3422
4,"(-111.891, 40.7608)",3887
5,"(-121.6784, 45.5152)",19640
6,"(-74.006, 40.7128)",3920
7,"(-94.2102, 36.3724)",21410
8,"(-73.5673, 45.5017)",1768
9,"(-87.6298, 41.8781)",3673


In [115]:
from geog import distance
import osmnx as ox

ox.config(use_cache=True, log_console=True)

# compute driving distance to closest airport
airport_distances = []
for location_with_closest_airport in locations_with_closest_airport.itertuples():
    location_point = location_with_closest_airport.location
    airport_id = location_with_closest_airport.closest_airport_id
    nearest_airport = filtered_airports_gpd.query(f"id == {airport_id}")
    airport_distances.append(
        distance(location_point, (float(nearest_airport.geometry.x), float(nearest_airport.geometry.y))) / 1000
    )

locations_with_closest_airport["distance_to_airport"] = airport_distances
locations_with_closest_airport

2022-05-08 09:59:48 Configured OSMnx 1.1.2
2022-05-08 09:59:48 HTTP response caching is on


,location,closest_airport_id,distance_to_airport
0,"(-77.1528, 39.085)",3483,27.743328
1,"(-74.6551, 40.3431)",3929,15.326738
2,"(-74.6551, 40.3431)",3929,15.326738
3,"(-71.0589, 42.3601)",3422,4.436499
4,"(-111.891, 40.7608)",3887,7.971425
5,"(-121.6784, 45.5152)",19640,41.228349
6,"(-74.006, 40.7128)",3920,15.949180
7,"(-94.2102, 36.3724)",21410,13.272572
8,"(-73.5673, 45.5017)",1768,11.850741
9,"(-87.6298, 41.8781)",3673,14.424408


In [137]:
# fan out for each location to ALL OTHER airports and plot over 100 mile grid
location_point = locations_with_closest_airport.head(1).location
airports_in_na = filtered_airports_gpd.query('iso_country in ("CA", "US")')
for airport_in_na in airports_in_na.itertuples():
    print(distance(location_point[0], (float(airport_in_na.geometry.x), float(airport_in_na.geometry.y))) / 1000)

5900.954472645225
567.1708142994191
1806.256281375436
1678.5073570939728
1652.3211164603506
4041.7093973334254
1017.5801874894037
1826.997434052289
2381.891148692298
1591.207136331167
1981.449421983573
1933.3477307417743
2129.8640576454764
3965.387022917787
3362.9638924841834
1324.0561448053768
4019.8979052905147
3160.1514827018536
3026.1762211950095
1140.0388581450256
3064.5273221021175
3926.1200023508522
2657.767634333977
2158.5750325833433
2732.942403305305
2577.861612663697
2121.373332794413
3169.2625595749287
2068.7089187492693
2223.9568951167225
3729.9769828701374
701.1460136809617
3826.712275218035
592.277798907031
3374.243500848926
1293.8036486653796
1317.1644440856894
1155.7014162984678
3940.1314590376537
3521.4573612620416
3735.4737075067696
2314.2861677157075
2844.937925060721
3527.4251645968757
4826.527687276311
4806.739662533933
3583.365988484252
1927.9388479083318
4464.771480390994
2224.1937638454406
1143.1614989391505
2247.8848633598836
3648.0183964385596
663.44141784361

In [5]:
# keep lowest travel time time in loop
# from airport compute distance to every other airport and driving distance to every 1km x 1km block in a 100 x 100 km grid
# add up all 1km x 1km grids and divide by lowest travel time (so it is a normalized ratio)
# plot all heights of 1km x 1km blocks